In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# Step.1 라이브러리 및 데이터 불러오기
- 라이브러리 버전 확인

In [67]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

print("success")

success


In [68]:
# 데이터 불러오기
    # PassengerId: 승객 번호, Transported: 전송
sample = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
# print(sample.head())
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
# print(test.head())
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
print(train.head())

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  


## 칼럼 별 설명:
- PassengerId: 각 승객의 고유 ID
- HomePlanet: 승객이 출발한 행성
- CryoSleep: 승객이 항해 기간 동안 애니메이션을 일시 중단하도록 선택했는지 여부를 나타냄
- Cabin: 승객이 머물고 있는 객실 번호입니다.
- Destination: 승객이 출발할 행성입니다.
- Age: 승객의 나이
- VIP: 승객이 항해 중 특별 VIP 서비스 비용을 지불했는지 여부.
- RoomService: 승객이 룸서비스에 대해 청구한 금액입니다.
- Name: 승객의 이름
- Transported: 승객이 다른 차원으로 운송되었는지 여부. 예측하려는 대상인 열

In [69]:
# 결측치 확인
test.info()
test.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


In [70]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [71]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


# Step.2 탐색적 자료 분석(EDA)
- 데이터 시각화
- 산점도, 막대 그래프 등
- 그래프 해석해서 설명을 달아야 함.
- 약간의 데이터 전처리

- 결측값들 개수 찾기

In [72]:
train_null = pd.DataFrame(train.isna().sum())
train_null= train_null.sort_values(by= 0)
print(train_null)

print()

test_null = pd.DataFrame(test.isna().sum())
test_null= test_null.sort_values(by= 0)
print(test_null)

                0
PassengerId     0
Transported     0
Age           179
RoomService   181
Destination   182
FoodCourt     183
Spa           183
VRDeck        188
Cabin         199
Name          200
HomePlanet    201
VIP           203
ShoppingMall  208
CryoSleep     217

                0
PassengerId     0
VRDeck         80
RoomService    82
HomePlanet     87
Age            91
Destination    92
CryoSleep      93
VIP            93
Name           94
ShoppingMall   98
Cabin         100
Spa           101
FoodCourt     106


In [73]:
fig = make_subplots(rows= 1,
                   cols= 2,
                   column_titles= ["train data", "test data"],
                   x_title= "Missing Values")

fig.add_trace(go.Bar(x= train_null[0], y= train_null.index,
                    orientation="h"),
                    1, 1)
              
fig.add_trace(go.Bar(x= test_null[0], y= test_null.index,
                    orientation="h"),
                    1, 2)          

fig.update_layout()

    - Transported와 PassengerId를 제외하고는 결측치가 존재함

# Step.3 데이터 전처리
- Feature Engineering
- 머신러닝(ML) 모형을 돌리기 위해 표준화 등 / 원핫-인코딩
- 파생변수(도출변수) 만들기
    - 왜 이 변수를 만들었는지에 대한 설명 필요

- 필요한 데이터를 제외하고는 제외

In [74]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [75]:
target = train['Transported']
target

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [76]:
print(train['PassengerId'].value_counts())  # x
# train['HomePlanet'].value_counts()  # o
# train['CryoSleep'].value_counts()  # o
print(train['Cabin'].value_counts())  # x
# train['Destination'].value_counts()  # o
# train['Age'].value_counts()  # o
# train['VIP'].value_counts()  # o
# train['RoomService'].value_counts()  # o
# train['FoodCourt'].value_counts()  # o
# train['ShoppingMall'].value_counts()  # o
# train['Spa'].value_counts()  # o
# train['VRDeck'].value_counts()  # o
print(train['Name'].value_counts())  # x

0001_01    1
6136_01    1
6141_01    1
6139_06    1
6139_05    1
          ..
3126_01    1
3124_03    1
3124_02    1
3124_01    1
9280_02    1
Name: PassengerId, Length: 8693, dtype: int64
G/734/S     8
G/109/P     7
B/201/P     7
G/1368/P    7
G/981/S     7
           ..
G/556/P     1
E/231/S     1
G/545/S     1
G/543/S     1
F/947/P     1
Name: Cabin, Length: 6560, dtype: int64
Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64


- 확인 해 본 결과 PassengerId, Cabin, Name의 특성은 예측에 영향을 안줄것으로 추정하여 제외하도록 함.

In [77]:
train_data = train.drop(['PassengerId', 'Cabin', 'Name'], axis= 1)
test_data = test.drop(['PassengerId', 'Cabin', 'Name'], axis= 1)
print(train_data.shape)
print(test_data.shape)

(8693, 11)
(4277, 10)


- 결측치를 최빈값으로 채우기 위해 각각의 데이터를 확인

In [78]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(4)
memory usage: 687.8+ KB


In [79]:
print(train_data["Age"].mode())
print(train_data["RoomService"].mode())
print(train_data["FoodCourt"].mode())
print(train_data["ShoppingMall"].mode())
print(train_data["Spa"].mode())
print(train_data["VRDeck"].mode())
print(train_data["HomePlanet"].mode())
print(train_data["CryoSleep"].mode())
print(train_data["Destination"].mode())
print(train_data["VIP"].mode())

0    24.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    Earth
dtype: object
0    False
dtype: object
0    TRAPPIST-1e
dtype: object
0    False
dtype: object


In [80]:
# float
train_data['Age'].replace(np.nan, 24.0, inplace= True)
train_data['RoomService'].replace(np.nan, 0.0, inplace= True)
train_data['FoodCourt'].replace(np.nan, 0.0, inplace= True)
train_data['ShoppingMall'].replace(np.nan, 0.0, inplace= True)
train_data['Spa'].replace(np.nan, 0.0, inplace= True)
train_data['VRDeck'].replace(np.nan, 0.0, inplace= True)
# object
train_data['HomePlanet'].replace(np.nan, 'Earth', inplace= True)
train_data['Destination'].replace(np.nan, 'TRAPPIST-1e', inplace= True)
train_data['CryoSleep'].replace(np.nan, False, inplace= True)
train_data['VIP'].replace(np.nan, False, inplace= True)

train_data.isna().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [81]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4190 non-null   object 
 1   CryoSleep     4184 non-null   object 
 2   Destination   4185 non-null   object 
 3   Age           4186 non-null   float64
 4   VIP           4184 non-null   object 
 5   RoomService   4195 non-null   float64
 6   FoodCourt     4171 non-null   float64
 7   ShoppingMall  4179 non-null   float64
 8   Spa           4176 non-null   float64
 9   VRDeck        4197 non-null   float64
dtypes: float64(6), object(4)
memory usage: 334.3+ KB


In [82]:
print(test_data["Age"].mode())
print(test_data["RoomService"].mode())
print(test_data["FoodCourt"].mode())
print(test_data["ShoppingMall"].mode())
print(test_data["Spa"].mode())
print(test_data["VRDeck"].mode())
print(test_data["HomePlanet"].mode())
print(test_data["CryoSleep"].mode())
print(test_data["Destination"].mode())
print(test_data["VIP"].mode())

0    18.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    0.0
dtype: float64
0    Earth
dtype: object
0    False
dtype: object
0    TRAPPIST-1e
dtype: object
0    False
dtype: object


In [83]:
# float
test_data['Age'].replace(np.nan, 18.0, inplace= True)
test_data['RoomService'].replace(np.nan, 0.0, inplace= True)
test_data['FoodCourt'].replace(np.nan, 0.0, inplace= True)
test_data['ShoppingMall'].replace(np.nan, 0.0, inplace= True)
test_data['Spa'].replace(np.nan, 0.0, inplace= True)
test_data['VRDeck'].replace(np.nan, 0.0, inplace= True)
# object
test_data['HomePlanet'].replace(np.nan, 'Earth', inplace= True)
test_data['Destination'].replace(np.nan, 'TRAPPIST-1e', inplace= True)
test_data['CryoSleep'].replace(np.nan, False, inplace= True)
test_data['VIP'].replace(np.nan, False, inplace= True)

test_data.isna().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

### 결측값 제거 성공!

- 원핫인코딩

In [84]:
train_data['Transported'] = train_data['Transported'].map({True: 1, False: 0})
categorical = ['HomePlanet', 'Destination', 'CryoSleep', 'VIP']

transformer = make_column_transformer(
(OneHotEncoder(), categorical),
remainder = 'passthrough')


train_transformed = transformer.fit_transform(train_data[categorical])
train_transformed_df = pd.DataFrame(train_transformed, columns= transformer.get_feature_names_out())
train_data = pd.concat([train_data, train_transformed_df], axis= 1)
train_data = train_data.drop(categorical, axis= 1)

test_transformed = transformer.fit_transform(test_data[categorical])
test_transformed_df = pd.DataFrame(test_transformed, columns= transformer.get_feature_names_out())
test_data = pd.concat([test_data, test_transformed_df], axis= 1)
test_data = test_data.drop(categorical, axis= 1)

print("success")

success


In [85]:
# train_data.info()
# test_data.info()

# Step.4 머신러닝 모형 개발
- 모형에 대한 설명 필요
- 모형 1~2개 사용 권장
- 교차 검증
- 하이퍼 파라미터 튜닝
    - 랜덤서치(매개변수(max_depth))
    - 그래드서치

- 독립변수와 종속변수를 구분
    - 독립변수: x
    - 종속변수: Transported == y

In [86]:
x_cols = test_data.columns
x = train_data[x_cols].to_numpy()
y = train_data['Transported'].to_numpy()

- 훈련데이터를 검증데이터로 분리
    - 검증데이터: val

In [87]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size= 0.3, random_state= 42)
x_train.shape,x_val.shape,y_train.shape,y_val.shape

((6085, 16), (2608, 16), (6085,), (2608,))

- 랜덤서치

In [88]:
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25),}
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state= 42), params,  # n_iter: 파라미터 검색 횟수
                        n_iter= 100, n_jobs= -1, random_state= 42)  # n_jobs: cpu코어 수

gs.fit(x_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f470c995790>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f470c9959d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f470c77a410>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f470cad4f50>},
                   random_state=42)

In [89]:
print(gs.best_params_)

{'max_depth': 28, 'min_impurity_decrease': 0.000404781258158029, 'min_samples_leaf': 6, 'min_samples_split': 9}


In [90]:
print(np.max(gs.cv_results_['mean_test_score']))

0.7878389482333608


In [91]:
dt = gs.best_estimator_
print(dt.score(x_val, y_val))

0.781058282208589


- LightGBM 사용

In [92]:
from lightgbm import LGBMClassifier

# lgb = LGBMClassifier(random_state= 42)
# lgb

In [93]:
# lgb_params= {
#     'objective': 'binary',
#     'n_estimators': 50,
#     'learning_rate': 0.08
# }

# lgb_predictions = 0
# lgb_scores= []
# lgb_fimp= []
# LGBM_features= list(train.columns)[:-1]
# # skf= StratifiedKFold(n_splits=FOLDS, shuffle= True, random_state=42)
# skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)

# for fold, (train_idx, valid_idx) in enumerate(skf.split(train[LGBM_features], train[target])):
#     print(f'\033][94m')
#     print(10*"=", f"Fold={fold+1}", 10*"=")
#     start_time= time.time()
    
#     x_train,x_val = train.iloc[train_idx][LGBM_features], train.iloc[valid_idx][LGBM_features]
#     y_train, y_val = train[target].iloc[train_idx], train[target].iloc[valid_idx]
    
#     lgb = LGBMClassifier(**lgb_params)
#     lgb.fit(X_train, y_train,verbose=0)
    
#     preds_valid = model.predict(X_valid)
#     acc = accuracy_score(y_valid,  preds_valid)
#     lgb_scores.append(acc)
#     run_time = time.time() - start_time
    
#     fim = pd.DataFrame(index=LGBM_FEATURES,
#                  data=model.feature_importances_,
#                  columns=[f'{fold}_importance'])
#     lgb_fimp.append(fim)
    
#     print(f"Fold={fold+1}, Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
#     test_preds = model.predict(test[LGBM_FEATURES]) 
#     lgb_predictions += test_preds/FOLDS
# print("")
# print("Mean Accuracy :", np.mean(lgb_scores))


# Step.5 모형 평가
- 훈련데이터를 쪼개어 훈련데이터 + 검증데이터 분리
- 정확도 비교
- 혼동행렬(confusion Matrix) 설명

# Step.6 제출
- 제출 양식 샘플은 만들어줌.

# 참고
- 다른 사람의 code 설명을 쭉 따라치는경우
    - 노트북 표절 방지 위해서, 참조한 코드는 반드시 링크 걸어둘것
    - 저자 이름, 글 제목, 링크 주소

# 마감일
- 4월 12일 17시 40분
- 제출 형태
    - Leaderboard 랭킹 사진 캡처
    - 고용노동부 보고 양식 (다음주에 확인하고 알려주실 예정)

# 정리
- value_counts(): 값 별 개수 세기
- Pandas라이브러리
    - fillna(): 결측값을 지정한 값으로 채움